# Extracción de Datos desde Múltiples Fuentes

## Importar bibliotecas

In [1]:
import pandas as pd
import sqlite3
import json
import requests

In [2]:
# crear carpeta para los resultados
import os

ruta_archivos = "../resultados/resultados_dia_2/"
if not os.path.exists(ruta_archivos):
    os.mkdir(ruta_archivos)
    print("Carpeta creada")
else:
    print("La carpeta ya existe")

La carpeta ya existe


## Crear datos de ejemplo en diferentes formatos

### Crear CSV

In [3]:
ventas_csv = pd.DataFrame({
    'id_venta': range(1, 6),
    'producto': ['Laptop', 'Mouse', 'Teclado', 'Monitor', 'Audífonos'],
    'precio': [1200, 25, 80, 300, 150]
})
ventas_csv.to_csv(ruta_archivos + 'ventas.csv', index=False)

### Crear Excel con múltiples hojas

In [4]:
clientes_df = pd.DataFrame({
    'id_cliente': [1, 2, 3],
    'nombre': ['Ana', 'Carlos', 'María'],
    'ciudad': ['Madrid', 'Barcelona', 'Valencia']
})

with pd.ExcelWriter(ruta_archivos + 'datos.xlsx') as writer:
    ventas_csv.to_excel(writer, sheet_name='Ventas', index=False)
    clientes_df.to_excel(writer, sheet_name='Clientes', index=False)

### Crear JSON

In [5]:
productos_json = [
    {'id': 101, 'nombre': 'Laptop', 'categoria': 'Electrónica'},
    {'id': 102, 'nombre': 'Mouse', 'categoria': 'Accesorios'}
]
with open(ruta_archivos + 'productos.json', 'w') as f:
    json.dump(productos_json, f)

### Crear base de datos SQLite

In [6]:
conn = sqlite3.connect(ruta_archivos + 'ventas.db')
pedidos_df = pd.DataFrame({
    'id_pedido': [1, 2, 3],
    'id_cliente': [1, 2, 1],
    'fecha': ['2024-01-15', '2024-01-16', '2024-01-17'],
    'total': [1225, 25, 380]
})
pedidos_df.to_sql('pedidos', conn, index=False, if_exists='replace')
conn.close()

## Extraer desde cada fuente

### Desde CSV

In [7]:
df_csv = pd.read_csv(ruta_archivos + 'ventas.csv')
print("Desde CSV:")
print(df_csv.head())

Desde CSV:
   id_venta   producto  precio
0         1     Laptop    1200
1         2      Mouse      25
2         3    Teclado      80
3         4    Monitor     300
4         5  Audífonos     150


### Desde Excel (hoja específica)

In [8]:
df_excel_ventas = pd.read_excel(ruta_archivos + 'datos.xlsx', sheet_name='Ventas')
df_excel_clientes = pd.read_excel(ruta_archivos + 'datos.xlsx', sheet_name='Clientes')
print("\nDesde Excel - Ventas:")
print(df_excel_ventas.head())


Desde Excel - Ventas:
   id_venta   producto  precio
0         1     Laptop    1200
1         2      Mouse      25
2         3    Teclado      80
3         4    Monitor     300
4         5  Audífonos     150


### Desde JSON

In [9]:
df_json = pd.read_json(ruta_archivos + 'productos.json')
print("\nDesde JSON:")
print(df_json)


Desde JSON:
    id  nombre    categoria
0  101  Laptop  Electrónica
1  102   Mouse   Accesorios


### Desde SQLite

In [10]:
conn = sqlite3.connect(ruta_archivos + 'ventas.db')
df_sql = pd.read_sql('SELECT * FROM pedidos', conn)
conn.close()
print("\nDesde SQLite:")
print(df_sql)


Desde SQLite:
   id_pedido  id_cliente       fecha  total
0          1           1  2024-01-15   1225
1          2           2  2024-01-16     25
2          3           1  2024-01-17    380


In [11]:
# con context manager
with sqlite3.connect(ruta_archivos + 'ventas.db') as conn2:
    df_sql2 = pd.read_sql('SELECT * FROM pedidos', conn2)
print("\nDesde SQLite con context manager:")
print(df_sql2)


Desde SQLite con context manager:
   id_pedido  id_cliente       fecha  total
0          1           1  2024-01-15   1225
1          2           2  2024-01-16     25
2          3           1  2024-01-17    380


## API simulada

In [12]:
# Simular API response
api_response = {
    'status': 'success',
    'data': [
        {'id': 201, 'producto': 'Webcam', 'stock': 15},
        {'id': 202, 'producto': 'Micrófono', 'stock': 8}
    ]
}

# Simular consumo de API
import json
df_api = pd.DataFrame(api_response['data'])
print("\nDesde API simulada:")
print(df_api)


Desde API simulada:
    id   producto  stock
0  201     Webcam     15
1  202  Micrófono      8


## API con requests

In [13]:
# API pública y gratuita para pruebas que no requiere autenticación

url = 'https://jsonplaceholder.typicode.com/posts/'

response = requests.get(url, timeout=30)

if response.status_code == 200:
    data = response.json()
    df_api = pd.DataFrame(data)
else:
    print(f"Error: {response.status_code}")

print("\nDesde API real:")
display(df_api)


Desde API real:


,userId,id,title,body
0,1,1,sunt aut facere repellat provident occaecati e...,quia et suscipit\nsuscipit recusandae consequu...
1,1,2,qui est esse,est rerum tempore vitae\nsequi sint nihil repr...
2,1,3,ea molestias quasi exercitationem repellat qui...,et iusto sed quo iure\nvoluptatem occaecati om...
3,1,4,eum et est occaecati,ullam et saepe reiciendis voluptatem adipisci\...
4,1,5,nesciunt quas odio,repudiandae veniam quaerat sunt sed\nalias aut...
...,...,...,...,...
95,10,96,quaerat velit veniam amet cupiditate aut numqu...,in non odio excepturi sint eum\nlabore volupta...
96,10,97,quas fugiat ut perspiciatis vero provident,eum non blanditiis soluta porro quibusdam volu...
97,10,98,laboriosam dolor voluptates,doloremque ex facilis sit sint culpa\nsoluta a...
98,10,99,temporibus sit alias delectus eligendi possimu...,quo deleniti praesentium dicta non quod\naut e...
